### Chamando o SQL serveless para rodar a query

In [0]:
%run /Workspace/Users/c00010492@cassi.com.br/Demandas/Reembolso/script_personalizado/sqlStatementSqlWarehouse

In [0]:
def executeScript(sql):
    sql_api = SQLStatementAPI()
    return sql_api.execute_sql(sql)

### Tabela: TB_avaliacao_appstore

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TB_avaliacao_appstore as

SELECT * FROM SILVER.WEBSCRAP01.TB_AVALIACAO_APPLESTORE
"""

### Tabela: TB_avaliacao_googleplay

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TB_avaliacao_googleplay as

SELECT * FROM SILVER.WEBSCRAP01.TB_AVALIACAO_GOOGLEPLAY
"""

### Tabela: TB_registro_login_usro

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TB_registro_login_usro as

SELECT * FROM ETL.RELPARTICIPANTE_MOBILE.TB_REGISTRO_LOGIN_USRO
"""

### Tabela: TD_participante

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TD_participante as


--TD_PARTICIPANTE
SELECT DISTINCT

PA.SK_TD_PARTICIPANTE,
ROUND(MONTHS_BETWEEN(NOW(), PA.DT_NASCIMENTO_BENEFICIARIO)/12) AS IDADE,
PA.DS_SEXO_BENEFICIARIO,
PA.NO_ESTADO_RESIDENCIAL,
PA.NO_MUNICIPIO_RESIDENCIAL,
PA.SG_UF_RESIDENCIAL,
PA.DS_ENDERECO_RESIDENCIAL,
PA.NU_LOGRADOURO_RESIDENCIAL,
PA.DS_TIPO_COMPL_RESIDENCIAL,
PA.DS_COMPLEMENTO_RESIDENCIAL,
PA.NO_BAIRRO_RESIDENCIAL,
PA.ST_TITULAR,
PA.CO_HD_BENEFICIARIO,
PA.DS_CONTRATO_BENEFICIARIO,
PA.NU_CPF_BENEFICIARIO

FROM GOLD.DW.TD_PARTICIPANTE PA
"""

### Tabela: TD_plano

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TD_plano as


SELECT DISTINCT
 PS.SK_TD_PLANO_SAUDE
,PS.CO_PLANO_SAUDE
,CASE WHEN PS.DS_PLANO_SAUDE IN ('RECIPROCIDADE I','RECIPROCIDADE II')
 THEN 'RECIPROCIDADE' ELSE PS.DS_PLANO_SAUDE
 END AS DS_PLANO_SAUDE
,CASE WHEN PS.DS_PLANO_SAUDE_CONTABIL IN ('RECIPROCIDADE I','RECIPROCIDADE II')
 THEN 'RECIPROCIDADE' ELSE PS.DS_PLANO_SAUDE_CONTABIL
 END AS DS_PLANO_SAUDE_CONTABIL

FROM GOLD.DW.TD_PLANO_SAUDE PS

WHERE PS.CO_PLANO_SAUDE NOT IN (-1,-2)

"""

### Tabela: TD_plano_s

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TD_plano_s as

SELECT DISTINCT
 PS.SK_TD_PLANO_SAUDE
,PS.CO_PLANO_SAUDE
,CASE WHEN PS.DS_PLANO_SAUDE IN ('RECIPROCIDADE I','RECIPROCIDADE II')
 THEN 'RECIPROCIDADE' ELSE PS.DS_PLANO_SAUDE
 END AS DS_PLANO_SAUDE
,CASE WHEN PS.DS_PLANO_SAUDE_CONTABIL IN ('RECIPROCIDADE I','RECIPROCIDADE II')
 THEN 'RECIPROCIDADE' ELSE PS.DS_PLANO_SAUDE_CONTABIL
 END AS DS_PLANO_SAUDE_CONTABIL

FROM GOLD.DW.TD_PLANO_SAUDE PS

WHERE PS.CO_PLANO_SAUDE NOT IN (-1,-2)
"""

### Tabela: TD_tempo

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TD_tempo as

SELECT DISTINCT
 TP.SK_TD_TEMPO
,TP.NU_MES
,TP.DS_MES
,TP.NU_ANO
, TP.DT_COMPLETA

FROM GOLD.DW.TD_TEMPO TP

WHERE TP.NU_ANO NOT IN (-1,-2,-3)
"""

### Tabela: TF_app_mobile

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TF_app_mobile as

SELECT
 TF.SK_TD_TEMPO_REGISTRO
,TF.SK_TD_PLANO_SAUDE
,TF.SK_TD_PARTICIPANTE
,TF.ST_CADASTRO_SOC
,MAX(TF.TS_CARGA) AS DT_ATUALIZACAO
,SUM(TF.QT_REGISTRO) AS QTD

FROM GOLD.DW.TF_APP_MOBILE_SAUDE TF
JOIN GOLD.DW.TD_PARTICIPANTE PA ON PA.SK_TD_PARTICIPANTE = TF.SK_TD_PARTICIPANTE

WHERE TF.ST_CADASTRO_SOC = 'S'
AND PA.DT_CANCELAMENTO_BENEFICIARIO IS NULL

GROUP BY
 TF.SK_TD_TEMPO_REGISTRO
,TF.SK_TD_PLANO_SAUDE
,TF.SK_TD_PARTICIPANTE
,TF.ST_CADASTRO_SOC
"""

### Tabela: TF_populacao

In [0]:
sql = """

create or replace table mldata.default.app_mobile_TF_populacao as

SELECT
TF.SK_TD_PLANO_SAUDE,
TF.SK_TD_TEMPO,
TF.SK_TD_PARTICIPANTE,
SUM(TF.QT_POPULACAO_ATIVA) QT_ATIVOS,
PA.NU_CPF_BENEFICIARIO

FROM GOLD.DW.TF_POPULACAO TF
INNER JOIN GOLD.DW.TD_PARTICIPANTE PA ON PA.SK_TD_PARTICIPANTE = TF.SK_TD_PARTICIPANTE

WHERE TF.ANO_MES_PROCESSAMENTO = DATE_FORMAT(ADD_MONTHS(CURRENT_DATE, - 1) , 'yyyyMM')

GROUP BY
TF.SK_TD_PLANO_SAUDE,
TF.SK_TD_TEMPO,
TF.SK_TD_PARTICIPANTE,
PA.NU_CPF_BENEFICIARIO

"""

### Tabela: Tamanho_Pop

In [0]:
sql = """

create or replace table mldata.default.app_mobile_Tamanho_Pop as

WITH TABELA_FINAL AS (

WITH TABELA AS (
SELECT
TF.SK_TD_PLANO_SAUDE,
TF.SK_TD_TEMPO,
TF.SK_TD_PARTICIPANTE,
SUM(TF.QT_POPULACAO_ATIVA) QT_ATIVOS,
PA.NU_CPF_BENEFICIARIO,
TP.DT_COMPLETA,
PA.SG_UF_RESIDENCIAL,
PS.DS_PLANO_SAUDE_CONTABIL,
ROUND(MONTHS_BETWEEN(NOW(), PA.DT_NASCIMENTO_BENEFICIARIO)/12) AS IDADE

FROM GOLD.DW.TF_POPULACAO                            TF
INNER JOIN GOLD.DW.TD_PARTICIPANTE                   PA ON PA.SK_TD_PARTICIPANTE = TF.SK_TD_PARTICIPANTE
JOIN GOLD.DW.TD_TEMPO                                TP ON TP.SK_TD_TEMPO = TF.SK_TD_TEMPO
JOIN GOLD.DW.TD_PLANO_SAUDE                          PS ON PS.SK_TD_PLANO_SAUDE = TF.SK_TD_PLANO_SAUDE

WHERE TF.ANO_MES_PROCESSAMENTO >= DATE_FORMAT(ADD_MONTHS(CURRENT_DATE, - 74) , 'yyyyMM')
AND PS.DS_PLANO_SAUDE_CONTABIL NOT LIKE "%RECIPROCIDADE%"

GROUP BY

TF.SK_TD_PLANO_SAUDE,
TF.SK_TD_TEMPO,
TF.SK_TD_PARTICIPANTE,
PA.NU_CPF_BENEFICIARIO,
TP.DT_COMPLETA,
PA.SG_UF_RESIDENCIAL,
PS.DS_PLANO_SAUDE_CONTABIL,
IDADE)

SELECT 
TABELA.SG_UF_RESIDENCIAL AS UF,
TABELA.DS_PLANO_SAUDE_CONTABIL AS PLANO,
TABELA.IDADE,
TABELA.DT_COMPLETA,
count(*) AS TOTAL_POP

FROM TABELA

GROUP BY 
TABELA.SG_UF_RESIDENCIAL,
TABELA.DS_PLANO_SAUDE_CONTABIL,
TABELA.IDADE,
TABELA.DT_COMPLETA)

SELECT *

FROM TABELA_FINAL
WHERE MONTH(TABELA_FINAL.DT_COMPLETA) = 12
   OR (MONTH(TABELA_FINAL.DT_COMPLETA) = MONTH(CURRENT_DATE()) -1 AND YEAR(TABELA_FINAL.DT_COMPLETA) = YEAR(CURRENT_DATE()))

"""